# Leemos la información

In [ ]:
#leemos los df
import pandas as pd
torneo_2024_1 = pd.read_csv('/content/drive/MyDrive/Clases_Innovador/torneo_2024_1.csv')
df_equipo_poder = pd.read_csv('/content/drive/MyDrive/Clases_Innovador/df_equipo_poder.csv')

In [ ]:
torneo_2024_1.drop(columns=['Unnamed: 0'], inplace=True)
torneo_2024_1

,Local,Visitante,Goles Local,Goles Visitante,Goles
0,Patriotas,Jaguares,0,1,1
1,Deportivo Pereira,Deportivo Cali,2,2,4
2,Once Caldas,Boyacá Chicó,2,1,3
3,Deportes Tolima,Fortaleza CEIF,1,2,3
4,América de Cali,Águilas Doradas,0,1,1
...,...,...,...,...,...
211,Once Caldas,Deportes Tolima,0,1,1
212,La Equidad,Once Caldas,1,2,3
213,Deportes Tolima,Santa Fe,1,2,3
214,Atlético Bucaramanga,Santa Fe,1,0,1


In [ ]:
df_equipo_poder.set_index('Equipo', inplace=True)
df_equipo_poder

,Goles_Favor,Goles_Contra
Equipo,,
Alianza F. C.,0.789474,1.526316
América de Cali,1.157895,0.842105
Atlético Bucaramanga,1.148148,0.592593
Atlético Nacional,1.105263,1.052632
Boyacá Chicó,1.157895,1.842105
Deportes Tolima,1.600000,0.960000
Deportivo Cali,1.263158,1.263158
Deportivo Pasto,0.789474,1.105263
Deportivo Pereira,1.400000,1.040000


In [ ]:
df_equipo_poder.loc['Águilas Doradas']

,Águilas Doradas
Goles_Favor,1.052632
Goles_Contra,1.000000


# Función de D. de Poisson

In [ ]:
from scipy.stats import poisson
prob = 0
def prediccion_puntos(equipo_local, equipo_visitante):
    #para definir lamda hago el producto entre las fortalezas y deilidades de cada equipo
    if equipo_local in df_equipo_poder.index and equipo_visitante in df_equipo_poder.index:
        #print(equipo_local, equipo_visitante)
        lambda_local = df_equipo_poder.at[equipo_local, 'Goles_Favor'] * df_equipo_poder.at[equipo_visitante, 'Goles_Contra']
        lambda_visitante = df_equipo_poder.at[equipo_visitante, 'Goles_Favor'] * df_equipo_poder.at[equipo_local, 'Goles_Contra']
        prob_local, prob_visitante, prob_empate = 0, 0, 0  #creo variables para guardar las probabilidades
        #número de goles del local
        for x in range(0,11):
            #número de goles del visitante
            for y in range(0,11):
                prob = poisson.pmf(x, lambda_local) * poisson.pmf(y, lambda_visitante)
                if x == y:
                    prob_empate += prob
                elif x > y:
                    prob_local += prob
                else:
                    prob_visitante += prob
            puntos_local = 3 * prob_local + prob_empate
            puntos_visitante = 3 * prob_visitante + prob_empate
            return (puntos_local, puntos_visitante)  #retornamos una tupla
    else:
        return 0, 0

# Traemos la Data del torneo actual

In [ ]:
ruta = 'https://es.wikipedia.org/wiki/Torneo_Finalizaci%C3%B3n_2024_(Colombia)'
tabla_fixture = pd.read_html(ruta)


In [ ]:
df_grupo_A = tabla_fixture[29]
df_grupo_B = tabla_fixture[33]


In [ ]:
df_grupo_A

,Pos.,Equipo,Pts.,PJ,G,E,P,GF,GC,Dif.,Unnamed: 10
0,1,Atlético Nacional,6,2,2,0,0,6,0,+6,NaN
1,2,Millonarios,6,2,2,0,0,3,1,+2,NaN
2,3,Santa Fe,0,2,0,0,2,0,6,−6,NaN
3,4,Deportivo Pasto,0,2,0,0,2,1,3,−2,NaN


## Hacemos limpieza de estas tablas

In [ ]:
df_grupo_A.drop(columns=['Unnamed: 10'], inplace=True)
df_grupo_B.drop(columns=['Unnamed: 10'], inplace=True)

In [ ]:
#vamos a modificar la columna Dif. para quitar espacios en blanco
df_grupo_A['Dif.'] = df_grupo_A['Dif.'].str.replace(' ', '')
df_grupo_B['Dif.'] = df_grupo_B['Dif.'].str.replace(' ', '')

In [ ]:
#vamos a quitar el +
df_grupo_A['Dif.'] = df_grupo_A['Dif.'].str.replace('+', '')
df_grupo_B['Dif.'] = df_grupo_B['Dif.'].str.replace('+', '')

In [ ]:
#vamos a cambiar los menos
df_grupo_A['Dif.'] = df_grupo_A['Dif.'].str.replace('−', '-')
df_grupo_B['Dif.'] = df_grupo_B['Dif.'].str.replace('−', '-')

In [ ]:
#convertimos la columna a int
df_grupo_A['Dif.'] = df_grupo_A['Dif.'].astype(int)
df_grupo_B['Dif.'] = df_grupo_B['Dif.'].astype(int)

In [ ]:
#cambiamos el nombre de la columna puntos
df_grupo_A.rename(columns={'Pts.': 'Puntos'}, inplace=True)
df_grupo_B.rename(columns={'Pts.': 'Puntos'}, inplace=True)

In [ ]:
df_grupo_A

,Pos.,Equipo,Puntos,PJ,G,E,P,GF,GC,Dif.
0,1,Atlético Nacional,6,2,2,0,0,6,0,6
1,2,Millonarios,6,2,2,0,0,3,1,2
2,3,Santa Fe,0,2,0,0,2,0,6,-6
3,4,Deportivo Pasto,0,2,0,0,2,1,3,-2


## Vamos a crear un diccionario de los grupos A y B

In [ ]:
#creamos una lista
lista_semis = ['Grupo A', 'Grupo B']
lista_semis

['Grupo A', 'Grupo B']

In [ ]:
#Creamos el dict con la lista
dict_semis = {'Grupo A': df_grupo_A, 'Grupo B': df_grupo_B}

In [ ]:
#recorremos el diccionario
for grupo in dict_semis:
    print(dict_semis[grupo])

   Pos.             Equipo  Puntos  PJ  G  E  P  GF  GC  Dif.
0     1  Atlético Nacional       6   2  2  0  0   6   0     6
1     2        Millonarios       6   2  2  0  0   3   1     2
2     3           Santa Fe       0   2  0  0  2   0   6    -6
3     4    Deportivo Pasto       0   2  0  0  2   1   3    -2
   Pos.           Equipo  Puntos  PJ  G  E  P  GF  GC  Dif.
0     1  Deportes Tolima       7   3  2  1  0   2   0     2
1     2           Junior       3   2  1  0  1   1   1     0
2     3      Once Caldas       2   2  0  2  0   1   1     0
3     4  América de Cali       1   3  0  1  2   1   3    -2


In [ ]:
#creamos un df a partir del diccionario
df_semis = pd.concat([dict_semis['Grupo A'], dict_semis['Grupo B']], ignore_index=True)
df_semis

,Pos.,Equipo,Puntos,PJ,G,E,P,GF,GC,Dif.
0,1,Atlético Nacional,6,2,2,0,0,6,0,6
1,2,Millonarios,6,2,2,0,0,3,1,2
2,3,Santa Fe,0,2,0,0,2,0,6,-6
3,4,Deportivo Pasto,0,2,0,0,2,1,3,-2
4,1,Deportes Tolima,7,3,2,1,0,2,0,2
5,2,Junior,3,2,1,0,1,1,1,0
6,3,Once Caldas,2,2,0,2,0,1,1,0
7,4,América de Cali,1,3,0,1,2,1,3,-2


In [ ]:
#guardamos este nuevo df
df_semis.to_csv('/content/drive/MyDrive/Clases_Innovador/df_semis.csv', index=False)

# Traemos el fixture de los cuadrangulares semifinales

In [ ]:
df_fix_grupo_A = tabla_fixture[31]
df_fix_grupo_B = tabla_fixture[35]

In [ ]:
#eliminamos columnas que no necesitamos
df_fix_grupo_A.drop(columns=df_fix_grupo_A.columns[0], inplace=True)
df_fix_grupo_B.drop(columns=df_fix_grupo_B.columns[0], inplace=True)

In [ ]:
#eliminamos de la 3 en adelante
df_fix_grupo_A.drop(columns=df_fix_grupo_A.columns[3:], inplace=True)
df_fix_grupo_B.drop(columns=df_fix_grupo_B.columns[3:], inplace=True)

In [ ]:
#concatenamos ambos cuadrangulares para que queden en un mismo registro
df_fix_semis = pd.concat([df_fix_grupo_A, df_fix_grupo_B], axis=0, ignore_index=True)
df_fix_semis

Resultados                             
                Local Resultado          Visitante
0         Millonarios     2 : 1    Deportivo Pasto
1   Atlético Nacional     5 : 0           Santa Fe
2     Deportivo Pasto     0 : 1  Atlético Nacional
3            Santa Fe     0 : 1        Millonarios
4     Deportivo Pasto       vs.           Santa Fe
5         Millonarios       vs.  Atlético Nacional
6            Santa Fe       vs.    Deportivo Pasto
7   Atlético Nacional       vs.        Millonarios
8   Atlético Nacional       vs.    Deportivo Pasto
9         Millonarios       vs.           Santa Fe
10    Deportivo Pasto       vs.        Millonarios
11           Santa Fe       vs.  Atlético Nacional
12        Once Caldas     0 : 0    Deportes Tolima
13             Junior     1 : 0    América de Cali
14    América de Cali     1 : 1        Once Caldas
15    Deportes Tolima     1 : 0             Junior
16    Deportes Tolima     1 : 0    América de Cali
17        Once Caldas       vs.             Junior
18    América de Cali       vs.    Deportes Tolima
19             Junior       vs.        Once Caldas
20             Junior       vs.    Deportes Tolima
21        Once Caldas       vs.    América de Cali
22    Deportes Tolima       vs.        Once Caldas
23    América de Cali       vs.             Junior

In [ ]:
#voy a quitar la columna de nivel 0
try:
    df_fix_semis.columns = df_fix_semis.columns.droplevel(0)
except:
    pass
df_fix_semis

,Local,Resultado,Visitante
0,Millonarios,2 : 1,Deportivo Pasto
1,Atlético Nacional,5 : 0,Santa Fe
2,Deportivo Pasto,0 : 1,Atlético Nacional
3,Santa Fe,0 : 1,Millonarios
4,Deportivo Pasto,vs.,Santa Fe
5,Millonarios,vs.,Atlético Nacional
6,Santa Fe,vs.,Deportivo Pasto
7,Atlético Nacional,vs.,Millonarios
8,Atlético Nacional,vs.,Deportivo Pasto
9,Millonarios,vs.,Santa Fe


In [ ]:
#cambiamos los valores de la columna resultado
df_fix_semis['Resultado'] = ''
df_fix_semis

,Local,Resultado,Visitante
0,Millonarios,,Deportivo Pasto
1,Atlético Nacional,,Santa Fe
2,Deportivo Pasto,,Atlético Nacional
3,Santa Fe,,Millonarios
4,Deportivo Pasto,,Santa Fe
5,Millonarios,,Atlético Nacional
6,Santa Fe,,Deportivo Pasto
7,Atlético Nacional,,Millonarios
8,Atlético Nacional,,Deportivo Pasto
9,Millonarios,,Santa Fe


In [ ]:
#vamos a crear una columna que se llame ganador
df_fix_semis['Ganador'] = ''
df_fix_semis

,Local,Resultado,Visitante,Ganador
0,Millonarios,,Deportivo Pasto,
1,Atlético Nacional,,Santa Fe,
2,Deportivo Pasto,,Atlético Nacional,
3,Santa Fe,,Millonarios,
4,Deportivo Pasto,,Santa Fe,
5,Millonarios,,Atlético Nacional,
6,Santa Fe,,Deportivo Pasto,
7,Atlético Nacional,,Millonarios,
8,Atlético Nacional,,Deportivo Pasto,
9,Millonarios,,Santa Fe,


In [ ]:
#guardamos el fixture de las semis
df_fix_semis.to_csv('/content/drive/MyDrive/Clases_Innovador/df_fix_semis.csv', index=False)

# Ejecutamos los partidos de las semifinales para verificar los ganadores de cada grupo (A, B)

## Creamos la función para obtener el ganador

In [ ]:
def obtener_ganador(df_semifinal_partidos):
    for index, col in df_semifinal_partidos.iterrows():
        local, visitante = col['Local'], col['Visitante']
        puntos_local, puntos_visitante = prediccion_puntos(local, visitante)
        if puntos_local > puntos_visitante:
            ganador = local
        elif puntos_local < puntos_visitante:
            ganador = visitante
        else:
            ganador = 'Empate'
        df_semifinal_partidos.loc[index, 'Ganador'] = ganador
        df_semifinal_partidos.loc[index, 'Resultado'] = f'{puntos_local.round(2)} vs {puntos_visitante.round(2)}'
    return df_semifinal_partidos



In [ ]:
#observemos los resultados
obtener_ganador(df_fix_semis)

,Local,Resultado,Visitante,Ganador
0,Millonarios,0.1 vs 0.47,Deportivo Pasto,Deportivo Pasto
1,Atlético Nacional,0.14 vs 1.27,Santa Fe,Santa Fe
2,Deportivo Pasto,0.13 vs 1.05,Atlético Nacional,Atlético Nacional
3,Santa Fe,0.13 vs 0.62,Millonarios,Millonarios
4,Deportivo Pasto,0.16 vs 1.55,Santa Fe,Santa Fe
5,Millonarios,0.08 vs 0.56,Atlético Nacional,Atlético Nacional
6,Santa Fe,0.16 vs 0.45,Deportivo Pasto,Deportivo Pasto
7,Atlético Nacional,0.08 vs 0.84,Millonarios,Millonarios
8,Atlético Nacional,0.13 vs 0.63,Deportivo Pasto,Deportivo Pasto
9,Millonarios,0.13 vs 1.08,Santa Fe,Santa Fe


In [ ]:
df_fix_semis

,Local,Resultado,Visitante,Ganador
0,Millonarios,0.1 vs 0.47,Deportivo Pasto,Deportivo Pasto
1,Atlético Nacional,0.14 vs 1.27,Santa Fe,Santa Fe
2,Deportivo Pasto,0.13 vs 1.05,Atlético Nacional,Atlético Nacional
3,Santa Fe,0.13 vs 0.62,Millonarios,Millonarios
4,Deportivo Pasto,0.16 vs 1.55,Santa Fe,Santa Fe
5,Millonarios,0.08 vs 0.56,Atlético Nacional,Atlético Nacional
6,Santa Fe,0.16 vs 0.45,Deportivo Pasto,Deportivo Pasto
7,Atlético Nacional,0.08 vs 0.84,Millonarios,Millonarios
8,Atlético Nacional,0.13 vs 0.63,Deportivo Pasto,Deportivo Pasto
9,Millonarios,0.13 vs 1.08,Santa Fe,Santa Fe


In [ ]:
#guardemos el df
df_fix_semis.to_csv('/content/drive/MyDrive/Clases_Innovador/df_fix_semis_update.csv', index=False)

# Observemos los resultados en la tabla de posiciones

In [ ]:
#recorrer esos partidos de las semifinales y actualizar las tablas de posición
for grupo in dict_semis:
    equipo_en_grupo = dict_semis[grupo]['Equipo'].values
    ##Prueba - Dividir los df para ambos cuadrangulares
    #df_fix_semis_2 = df_fix_semis[df_fix_semis['Local'].isin(['Deportes Tolima', 'Junior', 'Once Caldas', 'América de Cali'])]
    #df_fix_semis_2
    df_fix_semis_2 = df_fix_semis[df_fix_semis['Local'].isin(equipo_en_grupo)]
    for index, col in df_fix_semis_2.iterrows():
        local, visitante = col['Local'], col['Visitante']
        puntos_local, puntos_visitante = prediccion_puntos(local, visitante)
        dict_semis[grupo].loc[dict_semis[grupo]['Equipo'] == local, 'Puntos'] += puntos_local
        dict_semis[grupo].loc[dict_semis[grupo]['Equipo'] == visitante, 'Puntos'] += puntos_visitante

    dict_semis[grupo] = dict_semis[grupo].sort_values('Puntos', ascending=False).reset_index()
    dict_semis[grupo] = dict_semis[grupo][['Equipo', 'Puntos']]
    dict_semis[grupo] = dict_semis[grupo].round(0)

<ipython-input-203-912b2dcce50e>:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[6.10100032]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_semis[grupo].loc[dict_semis[grupo]['Equipo'] == local, 'Puntos'] += puntos_local
<ipython-input-203-912b2dcce50e>:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.12413321]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_semis[grupo].loc[dict_semis[grupo]['Equipo'] == local, 'Puntos'] += puntos_local


In [ ]:
dict_semis['Grupo B']

,Equipo,Puntos
0,Deportes Tolima,10.0
1,Junior,5.0
2,Once Caldas,5.0
3,América de Cali,4.0


In [ ]:
#truco
#dict_semis['Grupo A']['Puntos'] = 0
#dict_semis['Grupo B']['Puntos'] = 0

# Organicemos el paso a la final

## Creamos el diccionario de la final

In [ ]:
dict_semis.keys()

dict_keys(['Grupo A', 'Grupo B'])

In [ ]:
#Creemos un diccionario con los ganadores de los grupos
dict_final = {}
for grupo in dict_semis:
    ganador_grupo = dict_semis[grupo].loc[0, 'Equipo']
    dict_final[grupo] = ganador_grupo
dict_final

{'Grupo A': 'Millonarios', 'Grupo B': 'Deportes Tolima'}

## Creamos el df de la final

In [ ]:
columnas = ['Local', 'Resultado', 'Visitante', 'Ganador']
filas = {'Local':[dict_final['Grupo A'], dict_final['Grupo B']], 'Resultado':['Final Ida', 'Final Vuelta'],
         'Visitante':[dict_final['Grupo B'], dict_final['Grupo A']]}
df_final_partidos = pd.DataFrame(filas, columns=columnas)
df_final_partidos

,Local,Resultado,Visitante,Ganador
0,Millonarios,Final Ida,Deportes Tolima,NaN
1,Deportes Tolima,Final Vuelta,Millonarios,NaN


# Creamos la función para obtener el ganador de la liga

In [ ]:
def obtener_ganador_final(df_final_partidos):
    for index, col in df_final_partidos.iterrows():
        local = col['Local']
        visitante = col['Visitante']
        puntos_local, puntos_visitante = prediccion_puntos(local, visitante)
        if puntos_local > puntos_visitante:
            ganador = local
        else:
            ganador = visitante
        df_final_partidos.loc[index, 'Ganador'] = ganador
        df_final_partidos.loc[index, 'Resultado'] = f'{puntos_local.round(2)} vs {puntos_visitante.round(2)}'
    return df_final_partidos



In [ ]:
#probamos la función de la final
obtener_ganador_final(df_final_partidos)

,Local,Resultado,Visitante,Ganador
0,Millonarios,0.05 vs 0.7,Deportes Tolima,Deportes Tolima
1,Deportes Tolima,0.05 vs 0.5,Millonarios,Millonarios


In [ ]:
#Guardemos el df
df_final_partidos.to_csv('/content/drive/MyDrive/Clases_Innovador/df_final_partidos.csv', index=False)